<a href="https://colab.research.google.com/github/cnilsen/s8Regressions/blob/master/python_notebooks/precipitation_data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from pathlib import Path
from datetime import datetime
from functools import partial

from google.oauth2 import service_account
import ee

import math
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
import pandas as pd
import geopandas as gpd
import pandas_gbq
import seaborn
import probscale

import rasterio
from rasterio import features

from shapely.geometry import Point, Polygon
from shapely import geometry
from shapely.ops import nearest_points
import geopandas
from whitebox import WhiteboxTools

import ipywidgets
from ipywidgets import widgets, Button, Layout, HBox, VBox, FloatText, Textarea, HTML, GridspecLayout, Label, TwoByTwoLayout
from IPython.display import display
import ipyleaflet
from ipyleaflet import Map, CircleMarker, GeoData, LayersControl, WidgetControl, basemaps, Map, Marker, MarkerCluster, GeoJSON
from sidecar import Sidecar


In [1]:
from google.oauth2 import service_account
import ee
ee.Authenticate()
ee.Initialize()

%matplotlib inline

EEException: ignored

In [ ]:
import folium
# Define a method for displaying Earth Engine image tiles on a folium map.
def add_ee_layer(self, ee_object, vis_params, name):
    
    try:    
        # display ee.Image()
        if isinstance(ee_object, ee.image.Image):    
            map_id_dict = ee.Image(ee_object).getMapId(vis_params)
            folium.raster_layers.TileLayer(
            tiles = map_id_dict['tile_fetcher'].url_format,
            attr = 'Google Earth Engine',
            name = name,
            overlay = True,
            control = True
            ).add_to(self)
        # display ee.ImageCollection()
        elif isinstance(ee_object, ee.imagecollection.ImageCollection):    
            ee_object_new = ee_object.mosaic()
            map_id_dict = ee.Image(ee_object_new).getMapId(vis_params)
            folium.raster_layers.TileLayer(
            tiles = map_id_dicaadt['tile_fetcher'].url_format,
            attr = 'Google Earth Engine',
            name = name,
            overlay = True,
            control = True
            ).add_to(self)
        # display ee.Geometry()
        elif isinstance(ee_object, ee.geometry.Geometry):    
            folium.GeoJson(
            data = ee_object.getInfo(),
            name = name,
            overlay = True,
            control = True
        ).add_to(self)
        # display ee.FeatureCollection()
        elif isinstance(ee_object, ee.featurecollection.FeatureCollection):  
            ee_object_new = ee.Image().paint(ee_object, 0, 2)
            map_id_dict = ee.Image(ee_object_new).getMapId(vis_params)
            folium.raster_layers.TileLayer(
            tiles = map_id_dict['tile_fetcher'].url_format,
            attr = 'Google Earth Engine',
            name = name,
            overlay = True,
            control = True
        ).add_to(self)
    
    except:
        print("Could not display {}".format(name))
    
# Add EE drawing method to folium.
folium.Map.add_ee_layer = add_ee_layer

# Add custom basemaps to folium
basemaps = {
    'Google Maps': folium.TileLayer(
        tiles = 'https://mt1.google.com/vt/lyrs=m&x={x}&y={y}&z={z}',
        attr = 'Google',
        name = 'Google Maps',
        overlay = True,
        control = True
    ),
    'Google Satellite': folium.TileLayer(
        tiles = 'https://mt1.google.com/vt/lyrs=s&x={x}&y={y}&z={z}',
        attr = 'Google',
        name = 'Google Satellite',
        overlay = True,
        control = True
    ),
    'Google Terrain': folium.TileLayer(
        tiles = 'https://mt1.google.com/vt/lyrs=p&x={x}&y={y}&z={z}',
        attr = 'Google',
        name = 'Google Terrain',
        overlay = True,
        control = True
    ),
    'Google Satellite Hybrid': folium.TileLayer(
        tiles = 'https://mt1.google.com/vt/lyrs=y&x={x}&y={y}&z={z}',
        attr = 'Google',
        name = 'Google Satellite',
        overlay = True,
        control = True
    ),
    'Esri Satellite': folium.TileLayer(
        tiles = 'https://server.arcgisonline.com/ArcGIS/rest/services/World_Imagery/MapServer/tile/{z}/{y}/{x}',
        attr = 'Esri',
        name = 'Esri Satellite',
        overlay = True,
        control = True
    )
}

In [ ]:
# Create a folium map object.
my_map = folium.Map(location=[40.33, -120.42], zoom_start=4, height=500)

# Add custom basemaps
basemaps['Google Maps'].add_to(my_map)
#basemaps['Google Satellite Hybrid'].add_to(my_map)



### Watershed information

In [ ]:
s8_watersheds = ee.FeatureCollection("users/cnilsen/s8_watersheds").select(
    ['Location_N'], ['Location']
    ).getInfo()
ws = s8_watersheds['features']

#spatial_data = ee.Image.cat(pm25,precip,imp,roofs,nighttimeLights,roads,traffic,aerosol_index,so2,hcho,co,ch4,no2)

In [ ]:
ws_df = pd.DataFrame(ws)
pd.DataFrame(ws_df['properties'])

,properties
0,{'Location': 'TFWFD1'}
1,{'Location': 'SEAI1S8D_OUT'}
2,{'Location': 'TAC001S8D_OF2'}
3,{'Location': 'TAC003S8D_OF2'}
4,{'Location': 'PIEHIRES_OUT'}
5,{'Location': 'PIELORES_OUT'}
6,{'Location': 'SNO_COM'}
7,{'Location': 'SNO_LDR'}
8,{'Location': 'SEAC1S8D_OUT'}
9,{'Location': 'POSOUTFALL_60'}


In [ ]:
# Add the elevation model to the map object.
my_map.add_ee_layer(ee.FeatureCollection("users/cnilsen/s8_watersheds"), {}, 'DEM')
my_map

# make a kernel for convolution 

In [ ]:
#gaussianKernel = ee.Kernel.gaussian(500,100, 'meters')

## Get independent variables
---

In [ ]:
#Function to reduce predictors to watersheds 
def reduce_predictors (predictors, regions=ws, scale=30, reducer = ee.Reducer.mean()): 
    ##convolve with kernel firts 
    #smoothed = predictors.reduceNeighborhood(ee.Reducer.median(),gaussianKernel)
    regionReduce = predictors.reduceRegions(
        regions, reducer, scale).getInfo()
    lst = list(pd.DataFrame(regionReduce['features'])['properties'])
    df = (pd.DataFrame(lst))
    return df

In [ ]:
#Function to reduce predictors to watersheds 
#def reduce_predictors (predictors=None, regions=ws,scale= 10, reducer = ee.Reducer.mean()): 
#   regionReduce = predictors.reduceRegions(
#      regions, reducer, scale).getInfo()
# return regionReduce

### Land use categories 

In [ ]:
lu = ee.Image("users/cnilsen/lu_5m_pyramid") 

generalizedLU = lu.remap(
  [11, 12,  15,  18, 19,                            #Residential
  21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 39, #Industrial
  41, 42, 43, 44, 45, 46, 47, 48, 49,                             #Transportation 
  13, 14, 16, 17, 51, 52, 53, 54, 55, 56, 57, 58, 59, 61, 62, 63, 64, 65, 66, 67, 68, 69,             #Commercial
  81, 82, 83,                                                     #Agricultural 
  84, 85, 87, 88, 89,                 #Timber and Resource Extraction
  93,                                 #water
  71, 72, 74, 75, 76, 91, 92, 94, 95],                       #open space 
  
  [1, 1, 1, 1, 1,  
  2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, #Industrial
  3, 3, 3, 3, 3, 3, 3, 3, 3,                             #Transportation 
  4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4,              #Commercial
  5, 5, 5,                                                     #Agricultural 
  6, 6, 6, 6, 6,                 #Timber a nd Resource Extraction
  7,                                 #water
  8, 8, 8, 8, 8, 8, 8, 8, 8]  #open space
);  

In [ ]:
RES = generalizedLU.eq(1).rename('RES')
COM = generalizedLU.eq(4).rename('COM')
IND = generalizedLU.eq(2).rename('IND')
OPEN = generalizedLU.eq(8).rename('OPEN')
TIMBER = generalizedLU.eq(6).rename('TIMBER')
AG = generalizedLU.eq(5).rename('AG')
lu_predictors = ee.Image.cat(RES,COM,IND, OPEN, TIMBER, AG)
lu_results = reduce_predictors(lu_predictors)



In [ ]:
lu_results

,AG_median,COM_median,IND_median,Location,OPEN_median,RES_median,TIMBER_median
0,0.000000,0.126316,0.001514,TFWFD1,0.072211,0.728752,0
1,0.000000,0.401283,0.005879,SEAI1S8D_OUT,0.076011,0.276459,0
2,0.000000,0.395399,0.000000,TAC001S8D_OF2,0.229214,0.051623,0
3,0.000000,0.253283,0.024619,TAC003S8D_OF2,0.194199,0.253878,0
4,0.027772,0.030686,0.000000,PIEHIRES_OUT,0.082669,0.782145,0
5,0.000000,0.000000,0.000000,PIELORES_OUT,0.429956,0.570044,0
6,0.000000,0.834368,0.000000,SNO_COM,0.000000,0.161085,0
7,0.000000,0.229936,0.000000,SNO_LDR,0.000000,0.751345,0
8,0.000000,0.740724,0.000000,SEAC1S8D_OUT,0.023897,0.160948,0
9,0.000000,0.000000,0.000000,POSOUTFALL_60,0.009645,0.000000,0


### Get land cover predictors

In [ ]:
tncLC = ee.Image("users/jrobertson2000/psLandCover_1m_finPS_roofs")
imp = tncLC.eq(6).Or(tncLC.eq(7)).rename('impervious')
roofs = tncLC.eq(7).rename('roofs')

ee.Initialize()
pm25 = ee.Image("users/cnilsen/pm25clipped").rename('pm25')
precip = ee.Image("users/cnilsen/precip/PRISM800meter").rename('annual_precip')
nlcd = ee.Image('USGS/NLCD/NLCD2011').select(['percent_tree_cover'])

#nighttimeLights
nighttimeLights = ee.ImageCollection(
    'NOAA/DMSP-OLS/CALIBRATED_LIGHTS_V4').filter(
        ee.Filter.date('2009-01-01', '2013-12-31')).select('avg_vis').mean()

#onroad CO2
CO2 = ee.Image("users/stormwaterheatmap/CO2_onroad_2010")

#traffic
trafficReduced = ee.Image("users/cnilsen/trafficExport").reduceRegions(
    ee.FeatureCollection("TIGER/2016/Roads"), ee.Reducer.max(), 10)
#convert back to image
traffic = trafficReduced.reduceToImage(['max'], ee.Reducer.mean()).rename(
    'traffic')  ##//.select(['mean'],['avg_AADT'])
roadDensity = ee.Image(0).blend(traffic.gt(0))

#Slope
slope = ee.Terrain.slope("USGS/NED")

#Population
population = ee.Image("WorldPop/GP/100m/pop/USA_2010")

lc_predictors2 = ee.Image.cat(precip, imp, pm25, nlcd, slope, roofs, traffic,
                             roadDensity.rename('roadDensity'), CO2.rename('CO2'),
                             nighttimeLights.rename('nighttime_lights'),
                            population)
lc_results = reduce_predictors(predictors=lc_predictors2, scale=30)
lc_results


,CO2,Location,annual_precip,impervious,nighttime_lights,percent_tree_cover,pm25,population,roadDensity,roofs,slope,traffic
0,1.195606e+07,TFWFD1,1038.915485,0.423002,149.187004,11.418655,5.610534,10.887025,0.302576,0.140225,2.547214,34823.034963
1,8.272179e+06,SEAI1S8D_OUT,918.779082,0.380216,163.149635,27.366011,6.326861,3.422118,0.184472,0.116997,9.276244,45097.848451
2,2.861351e+07,TAC001S8D_OF2,997.383805,0.745441,269.164865,5.004187,6.366163,9.685268,0.479706,0.186282,5.740203,26564.190947
3,2.516035e+07,TAC003S8D_OF2,994.089275,0.670721,221.228687,8.039897,6.253202,5.066422,0.351091,0.128111,3.624301,45290.727021
4,3.185302e+05,PIEHIRES_OUT,1087.769673,0.233897,92.238345,25.272564,4.834875,5.800920,0.104136,0.117230,2.006226,6181.470789
5,1.879322e+05,PIELORES_OUT,1146.690367,0.052689,30.853020,64.014253,5.245596,1.128076,0.124665,0.025605,6.134781,1305.891337
6,6.147784e+06,SNO_COM,912.889195,0.630701,97.325241,9.312909,5.143090,35.627645,0.186241,0.241535,2.266982,15195.876352
7,8.789298e+05,SNO_LDR,1112.938907,0.244204,29.574629,17.469349,4.637970,3.163363,0.119012,0.072651,2.285916,3540.857261
8,6.519979e+06,SEAC1S8D_OUT,895.395438,0.768932,197.498829,7.399984,6.653779,60.693441,0.401975,0.390229,2.616628,58707.409080
9,6.420305e+05,POSOUTFALL_60,925.646964,0.918707,109.155057,3.580262,6.796271,9.792133,0.176033,0.020360,2.956093,979.249212


#### Road Density and Traffic

# slope

# population density

In [ ]:
lc_predictors.getInfo()

{'type': 'Image',
 'bands': [{'id': 'annual_precip',
   'data_type': {'type': 'PixelType', 'precision': 'double'},
   'dimensions': [7025, 3105],
   'crs': 'EPSG:4326',
   'crs_transform': [0.008333333332953738,
    0,
    -125.020833333,
    0,
    -0.008333333333011273,
    49.937499999]},
  {'id': 'impervious',
   'data_type': {'type': 'PixelType', 'precision': 'int', 'min': 0, 'max': 1},
   'dimensions': [1221926, 333864],
   'crs': 'EPSG:26910',
   'crs_transform': [1, 0, 13898, 0, -1, 5484376]},
  {'id': 'pm25',
   'data_type': {'type': 'PixelType', 'precision': 'float'},
   'dimensions': [886, 466],
   'crs': 'EPSG:4326',
   'crs_transform': [0.01, 0, -126.765, 0, -0.010000000000000007, 50.515]},
  {'id': 'percent_tree_cover',
   'data_type': {'type': 'PixelType',
    'precision': 'int',
    'min': 0,
    'max': 255},
   'dimensions': [161190, 104424],
   'crs': 'PROJCS["Albers Conical Equal Area", \n  GEOGCS["NAD83", \n    DATUM["North_American_Datum_1983", \n      SPHEROID["GR

# population_density = population.divide(ee.Image.pixelArea().divide(100*100))
population_density_results =   reduce_predictors(population_density, scale = 100, reducer = ee.Reducer.mean())     
population_density_results = population_density_results.rename(columns={"mean": "population_density"}) 
population_density_results

## make code for export

In [ ]:
result = pd.merge(lc_results, traffic_out2, on = ['Location'])
#result2 = pd.merge( lc_results, population_density_results, on=['Location'])

#result = pd.merge(result1, result2, on = ['Location'])#,  on = ['Location'])
result


,Location,annual_precip_median,impervious_median,nighttime_lights_median,percent_tree_cover_median,roofs_median,avg_AADT,co2_onRoad
0,TFWFD1,1038.916051,0.299817,149.181787,9.106201,0.000000,28921.774831,1.190731e+07
1,SEAI1S8D_OUT,918.763377,0.254518,163.137756,23.347037,0.000000,46062.455033,7.937393e+06
2,TAC001S8D_OF2,997.384636,0.933801,269.220126,4.259416,0.000000,18330.213230,2.848503e+07
3,TAC003S8D_OF2,994.090987,0.722020,221.293212,6.087753,0.008635,36698.557176,2.504171e+07
4,PIEHIRES_OUT,1087.770306,0.059410,92.236283,22.183726,0.000000,6489.394147,3.190203e+05
5,PIELORES_OUT,1146.687798,0.000000,30.862841,69.097465,0.000000,1679.025137,1.890937e+05
6,SNO_COM,912.899259,0.841515,97.325241,8.867408,0.007161,13236.630968,6.246828e+06
7,SNO_LDR,1112.913857,0.000000,29.574629,15.159751,0.000000,5607.706317,8.822104e+05
8,SEAC1S8D_OUT,895.459527,0.929802,197.516498,5.475262,0.103929,36897.474632,6.178399e+06
9,POSOUTFALL_60,925.646964,0.994220,109.138710,4.881415,0.000000,2074.567813,6.364498e+05


In [ ]:
lc_results.to_csv('../R-scripts/WatershedRegression/data/spatialPredictors_conv.csv')# pathUsers/cnilsen/Documents/repos/stormwaterheatmap/R-scripts/WatershedRegression/data/spatialPredictors.csv

In [ ]:
 s8data = pd.read_csv("../R-scripts/WatershedRegression/data/S8_data.csv") 

C:\Users\cnilsen\AppData\Local\Continuum\anaconda3\envs\white-box\lib\site-packages\IPython\core\interactiveshell.py:3058: DtypeWarning: Columns (44) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [ ]:
merged = pd.merge(result, s8data,  left_on='Location', right_on='location_id')

In [ ]:
merged.to_csv('../R-scripts/WatershedRegression/data/spatialPredictors_conv.csv')# pathUsers/cnilsen/Documents/repos/stormwaterheatmap/R-s

#### Air Quality 
Copernicus 5p data 
source: https://developers.google.com/earth-engine/datasets/catalog/sentinel-5p

In [ ]:
#Get copernicus 5p data  
fromDate = '2018-10-01'
toDate = '2019-09-30'
aerosol_index = ee.ImageCollection('COPERNICUS/S5P/NRTI/L3_AER_AI').select('absorbing_aerosol_index').filterDate(fromDate,toDate).mean()
so2 = ee.ImageCollection('COPERNICUS/S5P/NRTI/L3_SO2').select('SO2_column_number_density').filterDate(fromDate,toDate).mean()   
hcho = ee.ImageCollection('COPERNICUS/S5P/NRTI/L3_HCHO').select('tropospheric_HCHO_column_number_density').filterDate(fromDate,toDate).mean()   
co = ee.ImageCollection('COPERNICUS/S5P/NRTI/L3_CO').select('CO_column_number_density').filterDate(fromDate,toDate).mean()
ch4 = ee.ImageCollection('COPERNICUS/S5P/NRTI/L3_CH4').select('CH4_column_volume_mixing_ratio_dry_air').filterDate(fromDate,toDate).mean()
no2 = ee.ImageCollection('COPERNICUS/S5P/NRTI/L3_NO2').select('tropospheric_NO2_column_number_density').filterDate(fromDate,toDate).mean()

In [ ]:
#air_quality_5p = ee.Image.cat(aerosol_index,so2,hcho,co,ch4,no2)

In [ ]:
#air_quality_results = reduce_predictors(air_quality_5p, ws, 500, ee.Reducer.mean())

In [ ]:

#air_quality_results


,CO_column_number_density,Location,SO2_column_number_density,absorbing_aerosol_index,tropospheric_HCHO_column_number_density,tropospheric_NO2_column_number_density
0,0.034283,TFWFD1,0.000201,-0.931852,0.000070,0.000075
1,0.034194,SEAI1S8D_OUT,0.000305,-0.986750,0.000079,0.000088
2,0.034405,TAC001S8D_OF2,0.000203,-0.958915,0.000073,0.000078
3,0.034532,TAC003S8D_OF2,0.000218,-0.934803,0.000070,0.000081
4,0.034044,PIEHIRES_OUT,0.000316,-0.989775,0.000075,0.000065
5,0.034108,PIELORES_OUT,0.000402,-1.146496,0.000056,0.000051
6,0.033723,SNO_COM,0.000114,-0.940628,0.000071,0.000057
7,0.034101,SNO_LDR,0.000160,-0.973365,0.000067,0.000052
8,0.033849,SEAC1S8D_OUT,0.000194,-0.996946,0.000065,0.000077
9,0.033921,POSOUTFALL_60,0.000145,-1.136100,0.000071,0.000066


In [ ]:
#population density 


#### combine predictors 

---

In [ ]:
result

,Location,mean,avg_AADT,annual_precip,impervious,nighttime_lights,percent_tree_cover,AG,COM,IND,OPEN,RES,TIMBER
0,TFWFD1,2.559693,37794.057664,1038.915485,48.600511,149.187004,11.418655,0.000995,0.162653,0.007569,0.129517,0.683055,0.000000
1,SEAI1S8D_OUT,9.327751,48020.457700,918.779082,45.749487,163.149635,27.366011,0.000000,0.410781,0.058151,0.193319,0.337749,0.000000
2,TAC001S8D_OF2,5.768029,27825.232628,997.383805,73.846603,269.164865,5.004187,0.000000,0.444508,0.016786,0.303049,0.139692,0.000000
3,TAC003S8D_OF2,3.628419,53039.296743,994.089275,68.958385,221.228687,8.039897,0.000000,0.285303,0.058468,0.259673,0.235798,0.005241
4,PIEHIRES_OUT,1.989798,4170.059335,1087.769673,27.172705,92.238345,25.272564,0.049735,0.046266,0.000000,0.174651,0.700381,0.000000
5,PIELORES_OUT,6.266608,1433.454442,1146.690367,5.548221,30.853020,64.014253,0.000000,0.000000,0.000000,0.463160,0.533848,0.000000
6,SNO_COM,2.282610,15966.696203,912.889195,69.284034,97.325241,9.312909,0.000000,0.839360,0.000000,0.000000,0.159594,0.000000
7,SNO_LDR,2.306188,3716.912811,1112.938907,21.033720,29.574629,17.469349,0.000000,0.231284,0.000000,0.007713,0.761003,0.000000
8,SEAC1S8D_OUT,2.593381,60158.943068,895.395438,77.944339,197.498829,7.399984,0.000000,0.668761,0.000000,0.102203,0.228024,0.000000
9,POSOUTFALL_60,2.945336,973.420719,925.646964,45.978072,109.155057,3.580262,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000


## Temporal Predictors 

In [ ]:
## use precip from earth engine 
#ee.Initialize()
#day_range = ee.Date('2019-12-03').getRange('day','America/Los_Angeles');
#dataset = ee.ImageCollection('NASA/GPM_L3/IMERG_V06').filter(ee.Filter.date(day_range));
#
#precipitation = dataset.select('precipitationCal').sum();
#precipresults = reduce_predictors(precipitation,ws,100)
#p = list(pd.DataFrame(precipresults['features'])['properties'])
#p = (pd.DataFrame(p))
#p

## S8 data with timestamps 
import the data

In [ ]:
s8data_w_time = pd.read_csv("../R-scripts/WatershedRegression/data/s8_dataWithtime.csv") 

In [ ]:
s8data_w_time['Corrected start']
s8data_w_time['Corrected end']

0        2009-09-17T13:30
1        2009-09-17T13:30
2        2009-09-17T13:30
3        2009-09-17T13:30
4        2009-09-17T13:30
               ...       
46237    2010-10-09T22:33
46238    2010-10-09T22:33
46239    2010-10-09T22:33
46240    2010-10-09T22:33
46241    2010-10-09T22:33
Name: Corrected end, Length: 46242, dtype: object

In [ ]:
#make a small subset to start with 

s8 = s8data_w_time[s8data_w_time['Corrected duration'].gt(0)]
s8['Corrected start']
cols = list(s8[1:1])
cols
#loop through the data append to a new dataframe 

['sortID',
 'study_name',
 'sample_composite_flag',
 'sample_collection_method_description',
 'sample_id',
 'study_id',
 'type',
 'location_id',
 'lab_analysis_date',
 'result_lab_name',
 'sample_matrix',
 'season',
 'result_method',
 'sample_collection_method',
 'result_data_qualifier',
 'new_result_value',
 'original_eim_parameter_name',
 'new_result_units',
 'field_collection_start_time',
 'original_eim_result_value',
 'location_name',
 'sample_source',
 'access_id',
 'wetseason',
 'paramclass',
 'paramstatus',
 'field_collection_type',
 'lab_analysis_date_accuracy',
 'study_specific_location_id',
 'sample_replicate_flag',
 'field_collection_end_date',
 'field_collection_start_date',
 'parameter',
 'plot_data',
 'field_collection_end_time',
 'paramgroup',
 'original_eim_parameter_cas',
 'nondetect_flag',
 'field_collector',
 'lab_analysis_time',
 'result_method_description',
 'original_eim_units',
 'field_collection_comment',
 'fraction_analyzed',
 'result_additional_comment',
 'sto

In [ ]:
#get unique values of start-end, locations 
events =  s8[['location_id', 'x', 'y', 'Corrected start','Corrected end']]
s8_events = events.drop_duplicates().dropna()

# MODIS Land surface temperature

In [ ]:
#land surface temperature
ee.Initialize()
df = pd.DataFrame([])

for row in s8_events.itertuples():
    start_t = ee.Date(row[4],'America/Los_Angeles')
    end_t = ee.Date(row[5],'America/Los_Angeles')
    start_t_24hrs = start_t
    
    pt = ee.Geometry.Point(
       row[2],
        row[3]
    )
    #get the 30 min max precip for samplign duration 
    #temperature_collection = ee.ImageCollection('MODIS/006/MOD11A1').select('LST_Day_1km')
    temperature_collection = ee.ImageCollection('IDAHO_EPSCOR/GRIDMET').select('tmmx')
    
    #avgTemp = temperature_collection.filter(ee.Filter.date(start_t.advance(-7,'day'),end_t))
    monTemp = temperature_collection.filter(ee.Filter.date(start_t.advance(-30,'day'),end_t))
                                                #p_future24hrs = p_collection.filter(ee.Filter.date(start_t,end_t.advance(1,'day'))) #this is just a check to make sure the timezone assumptions are correct
   
    
    #avgTemp = avgTemp.mean().reduceRegions(pt,ee.Reducer.mean(),500).getInfo()
    monTemp = monTemp.mean().reduceRegions(pt,ee.Reducer.mean(),500).getInfo()
    
    
    #avgTemp = (list(avgTemp['features']))[0]['properties']['mean']
    monTemp = (list(monTemp['features']))[0]['properties']['mean']
  
    
    temp = pd.DataFrame ({'Location': row[1], 'start':row[4], 'end':row[5], 
                          'monTemp': monTemp},index=[0])
    df = df.append(temp,ignore_index=True)

In [ ]:
df
#monTemp['features'][0]['properties']['mean']

,Location,start,end,monTemp
0,KICLDRS8D_OUT,2011-07-12T21:00,2011-07-12T22:50,293.730286
1,SNO_COM,2011-09-25T05:25,2011-09-25T07:31,294.732880
2,SNO_HDR,2011-11-02T18:05,2011-11-02T20:40,286.806335
3,KICHDRS8D_OUT,2012-12-11T14:30,2012-12-11T17:30,283.258545
4,SNO_COM,2011-11-02T18:05,2011-11-02T21:15,286.465240
...,...,...,...,...
454,PIEHIRES_OUT,2013-04-04T09:00,2013-04-07T19:00,285.317261
455,SNO_LDR,2011-04-20T23:05,2011-04-24T10:40,285.206970
456,SNO_LDR,2012-06-12T13:35,2012-06-16T05:10,291.555267
457,SNO_LDR,2010-03-11T04:25,2010-03-14T23:10,285.595886


# Aerosol Thickness

In [ ]:

ee.Initialize()
df = pd.DataFrame([])

for row in s8_events.itertuples():
    start_t = ee.Date(row[4],'America/Los_Angeles')
    end_t = ee.Date(row[5],'America/Los_Angeles')
   
    
    pt = ee.Geometry.Point(
       row[2],
        row[3]
    )
    
    
    aerosol_depth = ee.ImageCollection('MODIS/006/MOD08_M3').select('Aerosol_Optical_Depth_Land_Ocean_Mean_Mean')
  
    monthly_aerosol_depth = aerosol_depth.filter(ee.Filter.date(start_t.advance(-40,'day'),end_t))
    monthly_aerosol_depth2 = monthly_aerosol_depth.mean().reduceRegions(pt,ee.Reducer.mean(),500).getInfo()
    
    #avgTemp = avgTemp.mean().reduceRegions(pt,ee.Reducer.mean(),500).getInfo()
    #monthly_aerosol_depth = monthly_aerosol_depth.mean().reduceRegions(pt,ee.Reducer.mean(),500).getInfo()
    
    
    #avgTemp = (list(avgTemp['features']))[0]['properties']['mean']
    returnedVal = (list(monthly_aerosol_depth2['features']))[0]['properties']
    
    if (bool(returnedVal)):
        monthly_aerosol_depth3 = returnedVal['mean']
    else: 
        monthly_aerosol_depth3 = -1
        
  
   
    temp_aerosol = pd.DataFrame ({'Location': row[1], 'start':row[4], 'end':row[5],
                                  'monthly_aerosol_depth': monthly_aerosol_depth3},index=[0])
    df = df.append(temp_aerosol,ignore_index=True)
    #print(monthly_aerosol_depth)

In [ ]:
df

,Location,start,end,monthly_aerosol_depth
0,KICLDRS8D_OUT,2011-07-12T21:00,2011-07-12T22:50,152.0
1,SNO_COM,2011-09-25T05:25,2011-09-25T07:31,172.0
2,SNO_HDR,2011-11-02T18:05,2011-11-02T20:40,169.0
3,KICHDRS8D_OUT,2012-12-11T14:30,2012-12-11T17:30,-1.0
4,SNO_COM,2011-11-02T18:05,2011-11-02T21:15,169.0
...,...,...,...,...
454,PIEHIRES_OUT,2013-04-04T09:00,2013-04-07T19:00,150.0
455,SNO_LDR,2011-04-20T23:05,2011-04-24T10:40,149.0
456,SNO_LDR,2012-06-12T13:35,2012-06-16T05:10,229.0
457,SNO_LDR,2010-03-11T04:25,2010-03-14T23:10,115.0


In [ ]:
#merge back to original s8[['location_id', 'x', 'y', 'Corrected start','Corrected end']]
new_df = pd.merge(s8[['access_id', 'location_id', 'Corrected start', 'Corrected end']], df,  how='left', left_on=['location_id','Corrected start', 'Corrected end'], right_on = ['Location','start','end'])

new_df.drop(['location_id', 'Corrected start',  'Corrected end'], axis=1)

new_df.to_csv("../R-scripts/WatershedRegression/data/monthly_aerosol_depth.csv")

In [ ]:
ee.Initialize()
df = pd.DataFrame([])

for row in s8_events.itertuples():
    start_t = ee.Date(row[4],'America/Los_Angeles')
    end_t = ee.Date(row[5],'America/Los_Angeles')
    start_t_24hrs = start_t
    #startdate = ee.Date(start_t)
    #print(start_t.getInfo())
    #ee.Date.parse(("M, yyyy");, date, timeZone)
    #get info from 
    #dummy coords 
    pt = ee.Geometry.Point(
       row[2],
        row[3]
    )
    #get the 30 min max precip for samplign duration 
    p_collection = ee.ImageCollection('NASA/GPM_L3/IMERG_V06').select('precipitationCal')
    p = p_collection.filter(ee.Filter.date(start_t,end_t))
    p_prev24hrs = p_collection.filter(ee.Filter.date(start_t.advance(-1,'day'),end_t))
    p_future24hrs = p_collection.filter(ee.Filter.date(start_t,end_t.advance(1,'day'))) #this is just a check to make sure the timezone assumptions are correct
    p_prev7days = p_collection.filter(ee.Filter.date(start_t.advance(-7,'day'),end_t))
    
    #get max and sum over the period 
    #max_p = p.max().reduceRegions(pt,ee.Reducer.first(),500).getInfo()
    #sum_p = p.sum().reduceRegions(pt,ee.Reducer.first(),500).getInfo()
    #max_p_prev24hrs = p_prev24hrs.max().reduceRegions(pt,ee.Reducer.first(),500).getInfo()
    #sum_p_prev24hrs = p_prev24hrs.sum().reduceRegions(pt,ee.Reducer.first(),500).getInfo()
    #sum_p_future24hrs =  p_future24hrs.sum().reduceRegions(pt,ee.Reducer.first(),500).getInfo()
    sum_p_prev7days =  p_prev7days.sum().reduceRegions(pt,ee.Reducer.mean(),30).getInfo()
    prev24hr = p_prev7days.sum().reduceRegions(pt,ee.Reducer.mean(),30).getInfo()
    prev48hr = p_prev7days.sum().reduceRegions(pt,ee.Reducer.mean(),30).getInfo()
    
    #get vals 
    #max_val = (list(max_p['features']))[0]['properties']['first']
    #sum_val = (list(sum_p['features']))[0]['properties']['first']
    #max_24hr = (list(max_p_prev24hrs['features']))[0]['properties']['first']
    #sum_24hr = (list(sum_p_prev24hrs['features']))[0]['properties']['first']
    #sum_future24hr = (list(sum_p_prev24hrs['features']))[0]['properties']['first']
    sum_7days = (list(sum_p_prev24hrs['features']))[0]['properties']['mean']
  
    
    temp = pd.DataFrame ({'Location': row[1], 'start':row[4], 'end':row[5], 
                         '7day_P':sum_7days },index=[0])
    df = df.append(temp,ignore_index=True)

KeyboardInterrupt: 

Daily precip

In [ ]:
gridmet = ee.ImageCollection("IDAHO_EPSCOR/GRIDMET").select('pr')

In [ ]:
df = pd.DataFrame([])
for row in s8_events.itertuples():
    start_t = ee.Date(row[4],'America/Los_Angeles')
    end_t = ee.Date(row[5],'America/Los_Angeles')
   
    pt = ee.Geometry.Point(
       row[2],
        row[3]
    )
    
    p_prev7days = gridmet.filter(ee.Filter.date(start_t.advance(-7,'day'),end_t))
    p24hr = gridmet.filter(ee.Filter.date(start_t.advance(-1,'day'),end_t))
    p48hr = gridmet.filter(ee.Filter.date(start_t.advance(-2,'day'),end_t))
    
    sum_p_prev7days =  p_prev7days.sum().reduceRegions(pt,ee.Reducer.mean(),30).getInfo()
    sum_24hr = p24hr.sum().reduceRegions(pt,ee.Reducer.mean(),30).getInfo()
    sum_48hr = p48hr.sum().reduceRegions(pt,ee.Reducer.mean(),30).getInfo()
    
    sum_p_prev7days = (list(sum_p_prev7days['features']))[0]['properties']['mean']
    sum_24hr = (list(sum_24hr['features']))[0]['properties']['mean']
    sum_48hr = (list(sum_48hr['features']))[0]['properties']['mean']
    
    temp = pd.DataFrame ({'Location': row[1], 'start':row[4], 'end':row[5], 
                          'sum_p_prev7days': sum_p_prev7days, '24hr_p':sum_24hr , '48hr_p':sum_48hr}, index=[0])
    df = df.append(temp,ignore_index=True)

In [ ]:
df

,Location,start,end,sum_p_prev7days,24hr_p,48hr_p
0,KICLDRS8D_OUT,2011-07-12T21:00,2011-07-12T22:50,0.525686,0.000000,0.000000
1,SNO_COM,2011-09-25T05:25,2011-09-25T07:31,3.389939,1.925879,1.925879
2,SNO_HDR,2011-11-02T18:05,2011-11-02T20:40,24.240530,6.946922,6.946922
3,KICHDRS8D_OUT,2012-12-11T14:30,2012-12-11T17:30,51.787702,10.564120,14.269490
4,SNO_COM,2011-11-02T18:05,2011-11-02T21:15,22.014960,7.669127,7.669127
...,...,...,...,...,...,...
454,PIEHIRES_OUT,2013-04-04T09:00,2013-04-07T19:00,61.847895,61.439263,61.439263
455,SNO_LDR,2011-04-20T23:05,2011-04-24T10:40,17.308345,5.993807,5.993807
456,SNO_LDR,2012-06-12T13:35,2012-06-16T05:10,31.364488,9.197854,9.197854
457,SNO_LDR,2010-03-11T04:25,2010-03-14T23:10,41.463340,27.686819,31.691529


In [ ]:
s8[['access_id', 'location_id', 'Corrected start', 'Corrected end']]

,access_id,location_id,Corrected start,Corrected end
1565,1115,KICLDRS8D_OUT,2011-07-12T21:00,2011-07-12T22:50
1567,1116,KICLDRS8D_OUT,2011-07-12T21:00,2011-07-12T22:50
1568,28515,KICLDRS8D_OUT,2011-07-12T21:00,2011-07-12T22:50
1569,28513,KICLDRS8D_OUT,2011-07-12T21:00,2011-07-12T22:50
1570,28556,KICLDRS8D_OUT,2011-07-12T21:00,2011-07-12T22:50
...,...,...,...,...
42283,62112,POT564S8D_OUT,2011-11-18T17:07,2011-11-22T12:09
42284,62100,POT564S8D_OUT,2011-11-18T17:07,2011-11-22T12:09
42285,62076,POT564S8D_OUT,2011-11-18T17:07,2011-11-22T12:09
42286,62102,POT564S8D_OUT,2011-11-18T17:07,2011-11-22T12:09


In [ ]:
#merge back to original s8[['location_id', 'x', 'y', 'Corrected start','Corrected end']]
new_df = pd.merge(s8[['access_id', 'location_id', 'Corrected start', 'Corrected end']], df,  how='left', left_on=['location_id','Corrected start', 'Corrected end'], right_on = ['Location','start','end'])

In [ ]:
new_df

,access_id,location_id,Corrected start,Corrected end,Location,start,end,sum_p_prev7days,24hr_p,48hr_p
0,1115,KICLDRS8D_OUT,2011-07-12T21:00,2011-07-12T22:50,KICLDRS8D_OUT,2011-07-12T21:00,2011-07-12T22:50,0.525686,0.000000,0.000000
1,1116,KICLDRS8D_OUT,2011-07-12T21:00,2011-07-12T22:50,KICLDRS8D_OUT,2011-07-12T21:00,2011-07-12T22:50,0.525686,0.000000,0.000000
2,28515,KICLDRS8D_OUT,2011-07-12T21:00,2011-07-12T22:50,KICLDRS8D_OUT,2011-07-12T21:00,2011-07-12T22:50,0.525686,0.000000,0.000000
3,28513,KICLDRS8D_OUT,2011-07-12T21:00,2011-07-12T22:50,KICLDRS8D_OUT,2011-07-12T21:00,2011-07-12T22:50,0.525686,0.000000,0.000000
4,28556,KICLDRS8D_OUT,2011-07-12T21:00,2011-07-12T22:50,KICLDRS8D_OUT,2011-07-12T21:00,2011-07-12T22:50,0.525686,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...
33292,62112,POT564S8D_OUT,2011-11-18T17:07,2011-11-22T12:09,POT564S8D_OUT,2011-11-18T17:07,2011-11-22T12:09,84.969600,60.188644,70.324075
33293,62100,POT564S8D_OUT,2011-11-18T17:07,2011-11-22T12:09,POT564S8D_OUT,2011-11-18T17:07,2011-11-22T12:09,84.969600,60.188644,70.324075
33294,62076,POT564S8D_OUT,2011-11-18T17:07,2011-11-22T12:09,POT564S8D_OUT,2011-11-18T17:07,2011-11-22T12:09,84.969600,60.188644,70.324075
33295,62102,POT564S8D_OUT,2011-11-18T17:07,2011-11-22T12:09,POT564S8D_OUT,2011-11-18T17:07,2011-11-22T12:09,84.969600,60.188644,70.324075


In [ ]:
new_df.to_csv("../R-scripts/WatershedRegression/data/7dayPrecip.csv")

do it again for just convective precip

---

## Precip intensity

In [ ]:
df_intensity = pd.DataFrame([])
for row in s8_events.itertuples():
    start_t = ee.Date(row[4],'America/Los_Angeles')
    end_t = ee.Date(row[5],'America/Los_Angeles')
   
    pt = ee.Geometry.Point(
       row[2],
        row[3]
    )
    
    p_prev7days = gridmet.filter(ee.Filter.date(start_t.advance(-7,'day'),end_t))
    p24hr = gridmet.filter(ee.Filter.date(start_t.advance(-1,'day'),end_t))
    p48hr = gridmet.filter(ee.Filter.date(start_t.advance(-2,'day'),end_t))
    
    max_p_prev7days =  p_prev7days.max().reduceRegions(pt,ee.Reducer.mean(),30).getInfo()
    max_24hr = p24hr.max().reduceRegions(pt,ee.Reducer.mean(),30).getInfo()
    max_48hr = p48hr.max().reduceRegions(pt,ee.Reducer.mean(),30).getInfo()
    
    max_p_prev7days = (list(max_p_prev7days['features']))[0]['properties']['mean']
    max_24hr = (list(max_24hr['features']))[0]['properties']['mean']
    max_48hr = (list(max_48hr['features']))[0]['properties']['mean']
    
    temp = pd.DataFrame ({'Location': row[1], 'start':row[4], 'end':row[5], 
                          'max_p_prev7days': max_p_prev7days, '24hr_p':max_24hr , '48hr_p':max_48hr}, index=[0])
    df_intensity = df_intensity.append(temp,ignore_index=True)    
    


In [ ]:
new_df_intensity = pd.merge(s8[['access_id', 'location_id', 'Corrected start', 'Corrected end']], df_intensity,  how='left', left_on=['location_id','Corrected start', 'Corrected end'], right_on = ['Location','start','end'])
new_df_intensity = new_df_intensity.rename(columns={"24hr_p	": "max_24hr_p", "48hr_p": "max_48hr_p"}) 

In [ ]:
new_df_intensity.to_csv("../R-scripts/WatershedRegression/data/precip_intensity.csv")